In [1]:
!pip install sentence-transformers faiss-cpu transformers accelerate langgraph >/dev/null 2>&1 || true

In [2]:
import json, faiss, torch, math
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from langgraph.graph import StateGraph, END
from typing import List, Any, TypedDict, Tuple

2025-11-08 14:27:08.915320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762612029.098574      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762612029.150546      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Inspect data & do preprocessing

In [3]:
# Data structure:

# "pre_text": the texts before the table;
# "post_text": the text after the table;
# "table": the table;
# "id": unique example id. composed by the original report name plus example index for this report. 

# "qa": {
#   "question": the question;
#   "program": the reasoning program;
#   "gold_inds": the gold supporting facts;
#   "exe_ans": the gold execution result;
#   "program_re": the reasoning program in nested format;
# }

# load in the train file
train_pth = '/kaggle/input/question-answering-financial-data/train.json'
with open(train_pth, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

dataset = []
for item in train_data:
    dataset.append({
        "id": item["id"],
        "pre_text": item["pre_text"],
        "post_text": item["post_text"],
        "table": item["table"],
        "question": item["qa"]["question"],
        "program": item["qa"]["program"],
        "gold_inds": item["qa"]["gold_inds"],
        "exe_ans": item["qa"]["exe_ans"],
        "program_re": item["qa"]["program_re"],
    })

df = pd.DataFrame(dataset)
print(df.shape)
df.head()

(6251, 9)


,id,pre_text,post_text,table,question,program,gold_inds,exe_ans,program_re
0,ADI/2009/page_49.pdf-1,[interest rate to a variable interest rate bas...,[fair value of forward exchange contracts afte...,"[[, october 31 2009, november 1 2008], [fair v...",what is the the interest expense in 2009?,"divide(100, 100), divide(3.8, #0)",{'text_1': 'if libor changes by 100 basis poin...,3.8,"divide(3.8, divide(100, 100))"
1,ABMD/2012/page_75.pdf-1,"[abiomed , inc ., and subsidiaries notes to co...",[the remaining unrecognized compensation expen...,"[[, number of shares ( in thousands ), weighte...","during the 2012 year , did the equity awards i...","multiply(607, 18.13), multiply(#0, const_1000)...",{'table_2': 'the granted of number of shares (...,yes,"greater(multiply(multiply(607, 18.13), const_1..."
2,AAL/2018/page_13.pdf-2,[the following table shows annual aircraft fue...,"[as of december 31 , 2018 , we did not have an...","[[year, gallons, average priceper gallon, airc...",what was the total operating expenses in 2018 ...,"divide(9896, 23.6%)",{'table_1': 'year the 2018 of gallons is 4447 ...,41932.20339,"divide(9896, 23.6%)"
3,INTC/2013/page_71.pdf-4,[the fair value of our grants receivable is de...,"[in the third quarter of 2013 , we sold our sh...","[[( in millions ), dec 282013, dec 292012], [a...",what percentage of total cash and investments ...,"divide(14001, 26302)",{'table_1': '( in millions ) the available-for...,0.53232,"divide(14001, 26302)"
4,ETR/2008/page_313.pdf-3,"[entergy louisiana , llc management's financia...",[the retail electric price variance is primari...,"[[, amount ( in millions )], [2007 net revenue...",what is the growth rate in net revenue in 2008?,"subtract(959.2, 991.1), divide(#0, 991.1)",{'table_1': 'the 2007 net revenue of amount ( ...,-0.03219,"divide(subtract(959.2, 991.1), 991.1)"


In [4]:
# Extract some questions
i = [415, 388, 104, 209]
qs = [(df.loc[q]["question"], df.loc[q]["id"].split('/')[0] ) for q in i]
questions = [f"For {i[1]}, {i[0]}" for i in qs]
questions
# These could be used for testing the RAG agent

['For HOLX, what is the estimated number of outstanding shares based in the stated eps?',
 'For AON, in 2015 what was the percentage change in the uncertain tax positions',
 'For CDNS, what is the total return if $ 1000000 are invested in nasdaq composite in 2009 and sold in 2010?',
 'For UNP, at december 31 , 2009 , what was the remaining compensation expense per share for the unvested awards?']

In [5]:
class TextPreprocessor:
    def __init__(self, chunk_size=200, overlap=50):
        self.chunk_size = chunk_size
        self.overlap = overlap
    
    def join_list_text(self, text_list):
        if isinstance(text_list, list):
            return " ".join(text_list)
        return text_list
    
    def table_to_text(self, table):
        if not table:
            return ""
        rows = []
        for row in table:
            rows.append(" | ".join([str(cell) for cell in row]))
        return "\n".join(rows)

    def chunk_text(self, text):
        words = text.split()
        chunks = []
        i = 0
        while i < len(words):
            chunk = " ".join(words[i:i + self.chunk_size])
            chunks.append(chunk)
            i += self.chunk_size - self.overlap
        return chunks

    def transform(self, df):
        """Takes original FinQA dataframe, returns chunk dataframe for embeddings"""
        
        # Preprocess text fields
        df["pre_text_str"] = df["pre_text"].apply(self.join_list_text)
        df["post_text_str"] = df["post_text"].apply(self.join_list_text)
        df["full_text"] = df["pre_text_str"] + " " + df["post_text_str"]
        df["table_text"] = df["table"].apply(self.table_to_text)

        # Build chunk list
        all_chunks = []
        for _, row in df.iterrows():
            text_chunks = self.chunk_text(row["full_text"])
            table_chunks = self.chunk_text(row["table_text"]) if row["table_text"] else []

            for chunk in text_chunks:
                all_chunks.append({
                    "doc_id": row["id"],
                    "source": "text",
                    "chunk": chunk,
                    "question": row["question"]
                })

            for chunk in table_chunks:
                all_chunks.append({
                    "doc_id": row["id"],
                    "source": "table",
                    "chunk": chunk,
                    "question": row["question"]
                })

        return pd.DataFrame(all_chunks)

In [6]:
preprocessor = TextPreprocessor(chunk_size=200, overlap=50)
chunks_df = preprocessor.transform(df)
print('chunks:', len(chunks_df))
chunks_df.head()

chunks: 36074


,doc_id,source,chunk,question
0,ADI/2009/page_49.pdf-1,text,interest rate to a variable interest rate base...,what is the the interest expense in 2009?
1,ADI/2009/page_49.pdf-1,text,relative to foreign currency exposures existin...,what is the the interest expense in 2009?
2,ADI/2009/page_49.pdf-1,text,counterparties . while the contract or notiona...,what is the the interest expense in 2009?
3,ADI/2009/page_49.pdf-1,text,unfavorable movement in foreign currency excha...,what is the the interest expense in 2009?
4,ADI/2009/page_49.pdf-1,text,a potential change in sales levels or local cu...,what is the the interest expense in 2009?


# Build Embeddings & FAISS indices

This part converts document chunks into vector embeddings using MiniLM, a light but effective transformer sentence-embedding model.

Components:
- Encoding every chunk into a dense vector
- Building a FAISS IndexFlatL2 search index
- Storing metadata
- Define FAISS retriever

In [7]:
# Load embedding model
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode chunks
embeddings = embedder.encode(
    chunks_df["chunk"].tolist(),
    batch_size=64,
    show_progress_bar=True
)
embeddings = np.array(embeddings).astype("float32")

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("Index size:", index.ntotal)

# Store metadata outside FAISS
metadata = chunks_df[["doc_id", "source", "question", "chunk"]].reset_index(drop=True)
docstore = metadata["chunk"].tolist()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/564 [00:00<?, ?it/s]

Index size: 36074


In [8]:
def faiss_search(query, k=20):
    """
    Search FAISS index and return top-k chunks with metadata.
    """
    q_emb = embedder.encode([query]).astype("float32") # make vector representation from the question
    distances, indices = index.search(q_emb, k) # search for the top-k nearest vectors

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx == -1:
            continue
        metadata_row = metadata.iloc[idx].to_dict()
        results.append({
            "chunk": metadata_row["chunk"],
            "doc_id": metadata_row["doc_id"],
            "source": metadata_row["source"],
            "distance": float(dist)
        })
    return results

# Test it on Q1
results = faiss_search(questions[0], k=4)
for r in results:
    print(f"Doc: {r['doc_id']}")
    print(f"Source: {r['source']}")
    print(f"Distance: {r['distance']:.4f}")
    print("Chunk:")
    print(r["chunk"])
    print("-" * 100)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Doc: ALLE/2018/page_121.pdf-2
Source: text
Distance: 0.8385
Chunk:
note 18 2013 earnings per share ( eps ) basic eps is calculated by dividing net earnings attributable to allegion plc by the weighted-average number of ordinary shares outstanding for the applicable period . diluted eps is calculated after adjusting the denominator of the basic eps calculation for the effect of all potentially dilutive ordinary shares , which in the company 2019s case , includes shares issuable under share-based compensation plans . the following table summarizes the weighted-average number of ordinary shares outstanding for basic and diluted earnings per share calculations: . at december 31 , 2018 , 0.1 million stock options were excluded from the computation of weighted-average diluted shares outstanding because the effect of including these shares would have been anti-dilutive . note 19 2013 net revenues net revenues are recognized based on the satisfaction of performance obligations under the terms 

#  Build LangGraph agent pipeline
This final chapter connects all components into a RAG pipeline.

Components:
- Flan-T5 LLM (text-to-text generator)
- Cross-encoder MiniLM reranker
- LangGraph workflow

In [9]:
# Load LLM
device = "cuda" if torch.cuda.is_available() else "cpu"
# model_id = "Qwen/Qwen1.5-1.8B-Chat"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

flan_model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(flan_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_id, device_map="auto")

# Load Reranker
rerank_model_id = "cross-encoder/ms-marco-MiniLM-L-6-v2"
rerank_tokenizer = AutoTokenizer.from_pretrained(rerank_model_id)
rerank_model = AutoModelForSequenceClassification.from_pretrained(rerank_model_id).to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

### LLM generator function

In [10]:
def build_prompt(question, context=None):
    system = (
        "You are a helpful financial analysis assistant. "
        "Use ONLY the provided context. "
        "Do NOT repeat the context. "
        "Do NOT provide any explanation. "
        "If the answer is not in the context, just say 'Not found in provided reports.'"
    )
    if context:
        return f"{system}\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
    return f"{system}\n\nQuestion: {question}\nAnswer:"

def generate_llm_answer(prompt: str, max_tokens: int = 256, temperature: float = 0.2, top_p: float = 0.9):
    """
    LLM generator function. Returns answer string.
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = decoded.replace(prompt, "").strip() # remove prompt
    return answer

In [11]:
class Reranker:
    """
    Cross-encoder based reranker that scores query/doc pairs.
    Return value: list of tuples (doc, score) sorted by decreasing relevance.
    """
    def __init__(self, model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2", device: str = None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        # CrossEncoder handles batching automatically
        self.model = CrossEncoder(model_name, device=self.device)

    def score(self, query: str, docs: List[str], batch_size: int = 16) -> List[float]:
        """Return raw scores for docs."""
        if not docs:
            return []
        pairs = [[query, d] for d in docs]
        scores = self.model.predict(pairs, batch_size=batch_size, show_progress_bar=False)
        return scores

    def top_k(self, query: str, docs: List[str], k: int = 4, batch_size: int = 16) -> List[Tuple[str, float]]:
        """Returns the best k (doc, score) pairs sorted high->low."""
        if not docs:
            return []
        scores = self.score(query, docs, batch_size=batch_size)
        doc_score_pairs = list(zip(docs, [float(s) for s in scores]))
        doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
        return doc_score_pairs[:k]


### LangGraph nodes

In [12]:
class State(TypedDict):
    question: str
    plan: str
    need_retrieval: bool
    retrieved_chunks: List[str]
    answer: str

def planner_node(state: State):
    """Decision maker for RAG application."""
    q = state["question"].lower()
    finance_keywords = [
        "revenue","income","profit","loss","net","year","earnings","balance",
        "$","million","billion","201","202"
    ]
    need = any(k in q for k in finance_keywords)
    plan = "RAG" if need else "Direct LLM"
    return {"plan": plan, "need_retrieval": need}

def retriever_node(state):
    """Retriever: fast FAISS search -> many noisy hits -> rerank later."""
    results = faiss_search(state["question"], k=20)
    chunks = [r["chunk"] for r in results]
    return {"retrieved_chunks": chunks}

def rerank_node(state):
    """Reranker: orders the chunks based on accuracy/confidence."""
    query = state["question"]
    docs = state.get("retrieved_chunks", [])
    ranked = reranker.top_k(query, docs, k=3)
    top_docs = [d for d, s in ranked]
    top_scores = [s for d, s in ranked]
    avg_conf = sum(top_scores) / len(top_scores) if top_scores else 0.0

    # If confidence is low, planner should try again
    if avg_conf < 0.15:
        print("Low confidence (avg:", avg_conf, ") — consider fallback.")
        return {"retrieved_chunks": top_docs, "retrieval_scores": top_scores, "low_confidence": True}

    return {"retrieved_chunks": top_docs, "retrieval_scores": top_scores, "low_confidence": False}

def generator_node(state):
    """Generate answer based on retrieved context chunks."""
    context = "\n\n".join(state.get("retrieved_chunks", []))
    q = state["question"]
    prompt = build_prompt(q, context if context else None)
    answer = generate_llm_answer(prompt)
    return {"answer": answer}


In [13]:
# Build LangGraph workflow
reranker = Reranker(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")
workflow = StateGraph(State)

# Register nodes
workflow.add_node("planner", planner_node)
workflow.add_node("retriever", retriever_node)
workflow.add_node("rerank", rerank_node)
workflow.add_node("generator", generator_node)

# Planner decides whether RAG is needed
workflow.set_entry_point("planner")
workflow.add_conditional_edges("planner", lambda s: "retriever" if s["need_retrieval"] else "generator")

# Normal RAG flow: retriever -> rerank -> generator
workflow.add_edge("retriever", "rerank")
workflow.add_edge("rerank", "generator")
workflow.add_edge("generator", END)

# Compile graph
app = workflow.compile()

README.md: 0.00B [00:00, ?B/s]

In [14]:
def rag_chat(query, debug=False):
    """Wrapper for RAG + LLM answer generation."""
    state = {"question": query}
    final = None

    for step in app.stream(state):
        final = step
        if debug:
            print("\n--- Step ---")
            print(step)

    final_state = list(final.values())[-1]
    return final_state.get("answer", "No answer generated.")

# Test it on the questions
for q in questions:
    print(f"Original q: {q}")
    print(f"Found answer: {rag_chat(q)}\n")

Original q: For HOLX, what is the estimated number of outstanding shares based in the stated eps?
Found answer: Not found in provided reports

Original q: For AON, in 2015 what was the percentage change in the uncertain tax positions


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found answer: Interest and penalties

Original q: For CDNS, what is the total return if $ 1000000 are invested in nasdaq composite in 2009 and sold in 2010?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found answer: Not found in provided reports

Original q: For UNP, at december 31 , 2009 , what was the remaining compensation expense per share for the unvested awards?
Found answer: Not found in provided reports



### Conclusions:
- Flan-T5-Large/Qwen: too small models for quality answers based on domain knowledge
- Tried Mistral 7-B, but I had version conflicts, that would take time to investigate